In [ ]:
# Stats on Ca2+ imaging with miniscope and Osc
#######################################################################################
                            # Define Experiment type #
#######################################################################################

DrugExperiment=1 # 1 if CGP Experiment, 0 if Baseline Experiment

AnalysisID='' 

Local=1

choosed_folder1='Osc_2025-05-04_10_45_30_likeAH' # for Baseline Expe
choosed_folder2='Osc_2025-05-04_10_45_30_likeAH' # for CGP Expe

choosed_folder3='VigSt_2025-05-03_10_01_32'

#######################################################################################
                                # Load packages #
#######################################################################################

import statsmodels.api as sm
import quantities as pq
import numpy as np
import math 
import neo
import json
from pathlib import Path
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, Cursor
import pickle
import os
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats import zscore
from scipy.stats import sem
from datetime import datetime
import shutil
from scipy import stats
from scipy.stats import t
from scipy.stats import norm

import warnings
warnings.filterwarnings("ignore")


def column_with_max_single_per_row(row):
    max_val = row.max()  # Find the max value in the row
    max_columns = row == max_val  # Boolean Series of columns with max value
    
    if max_columns.sum() > 1:
        return 'NoPref'  # More than one column has the max value
    else:
        return max_columns.idxmax()  # Return the name of the column with max value

def divide_keys(data):
    it=list(data.keys())[0]
    d=data[it]
    data[it]=d.replace(0, np.nan)
    for sheet in list(data.keys())[1:]: 
        data[sheet].div(data[it][sheet], axis=0)
    del data[it]
    return data    

########################################################################
                            # Define Directories #
########################################################################

# Specify the directory containing the Excel files
InitialDirectory1 = "//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/Analysed2025_AB/_baseline_analysis" if Local else "/crnldata/waking/audrey_hay/L1imaging/Analysed2025_AB/_baseline_analysis" 
directory1= f'{InitialDirectory1}/{choosed_folder1}'
InitialDirectory2 ="//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/Analysed2025_AB/_CGP_analysis" if Local else "/crnldata/waking/audrey_hay/L1imaging/Analysed2025_AB/_CGP_analysis"
directory2= f'{InitialDirectory2}/{choosed_folder2}'

directory3= f'{InitialDirectory1}/{choosed_folder3}'

# Get the current date and time
FolderNameSave=str(datetime.now())[:19]
FolderNameSave = FolderNameSave.replace(" ", "_").replace(".", "_").replace(":", "_")
destination_folder= f"//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/Analysed2025_AB/_global_analysis/AVG_Osc_{FolderNameSave}{AnalysisID}" if Local else f"/crnldata/waking/audrey_hay/L1imaging/Analysed2025_AB/_global_analysis/AVG_Osc_{FolderNameSave}{AnalysisID}"
os.makedirs(destination_folder)
folder_to_save=Path(destination_folder)

# Copy the script file to the destination folder
source_script = "C:/Users/Manip2/SCRIPTS/CodePythonAudrey/CodePythonAurelie/HayLabAnalysis/python/_MINI&OE_7_average_osc_per_cluster.ipynb" if Local else "/python/_MINI&OE_7_average_osc_per_cluster.ipynb" 
destination_file_path = f"{destination_folder}/_MINI&OE_7_average_osc_per_cluster.txt"
shutil.copy(source_script, destination_file_path)


#######################################################################################
                            # Do the average per cluster #
#######################################################################################

CTX=['', 'S1', 'PFC', 'S1PFC']
Coupling=['', 'UnCoupled', 'Coupled']
OscList=['Spdl', 'SWR']
Drugs= ['baseline', 'CGP'] if DrugExperiment else ['baseline']

NrSubtypeList=['L1NDNF_mice','L2_3_mice']


# Load Global Table 
with open(f'{directory3}/VigStates_Global_cluster.pkl', 'rb') as pickle_file:
    Vig_cluster_dfO = pickle.load(pickle_file)

for o, Osc in enumerate(OscList): 
    
    print(Osc, 'oscillations analysis...')

    AllOscStatut=pd.DataFrame()
    AllOscDuration=pd.DataFrame()
    AllGlobalSpindle=pd.DataFrame()
    AllOscStartLocation=pd.DataFrame()
    AllOscSWRinside=pd.DataFrame()
    AllOscCoupling=pd.DataFrame()

                
    # Initialize an empty list to store the dataframes
    combined_df = []
    filtered_df=[]

    ###########################################################################
                                ##### GLOBAL #####
    ###########################################################################

    # Load Global Table 
    with open(f'{directory2}/{Osc}_Global.pkl', 'rb') as pickle_file:
        combined_df = pickle.load(pickle_file)

    combined_df['Unique_Unit'] = combined_df['Unique_Unit'].astype(str)
    combined_df['UnitNumber'] = combined_df['UnitNumber'].astype(str)
    combined_df['UnitValue'] = combined_df['UnitValue'].astype(str)

    combined_df[f'{Osc}Statut'] = combined_df[f'{Osc}Statut'].astype(str)

    combined_df['Unit_ID'] = combined_df['Mice'] + combined_df['Unique_Unit']
    combined_df['Session_ID'] = combined_df['Mice'] + '_' + combined_df['Session_Date'] + '_' + combined_df['Session_Time']

    unique_count = combined_df['Unit_ID'].nunique()
    print(unique_count, f'Total neurons in the cross-registration') 
    
    combined_df[f'{Osc}_ID'] = combined_df['Mice'] + '_' + combined_df['Session_Date'] + '_' + combined_df['Session_Time'] + '_' +combined_df[f'{Osc}Number'].astype(str)
    
    unique_count = combined_df[f'{Osc}_ID'].nunique()
    print(unique_count, f'{Osc} recorded in total')

    writer = pd.ExcelWriter(f'{folder_to_save}/{Osc}_Global.xlsx')
    combined_df.to_excel(writer)
    writer.close()
    
    ###########################################################################
                                ##### GLOBAL #####
    ###########################################################################

    for NrSubtype in NrSubtypeList: 

        if NrSubtype=='L1NDNF_mice':
            MiceList=['BlackLines', 'BlueLines', 'GreenDots', 'GreenLines', 'RedLines']
        else:
            MiceList=['PurpleSquare', 'ThreeColDots', 'ThreeBlueCrosses']

        Data=['Ca', 'Sp']
        for data in Data:
            for drug in Drugs: 
                for ctx in CTX:
                    for coup in Coupling:  
                        #ctx= '' if Osc=='SWR' else ctx                      
                        locals()[f'{data}PSTH{Osc}_{coup}{ctx}{drug}']={}
                        dfsPSTH_per_sheet = locals()[f'{data}PSTH{Osc}_{coup}{ctx}{drug}']
                        nametofind2=f'{Osc}_{data}PSTH_{coup}{ctx}{drug}'
                        print(nametofind2)
                        # Recursively traverse the directory structure
                        for directory in [directory2]:
                            for root, _, files in os.walk(directory):
                                for filename in files:
                                    if filename.endswith('.pkl') and nametofind2 in filename: 
                                        if any(name in filename for name in MiceList): 
                                            # Construct the full path to the file
                                            filepath = os.path.join(root, filename)
                                            with open(filepath, 'rb') as pickle_file:
                                                df = pickle.load(pickle_file)
                                            for key, value in df.items():
                                                if key in dfsPSTH_per_sheet:
                                                    dfsPSTH_per_sheet[key]=pd.concat([pd.DataFrame(dfsPSTH_per_sheet[key]),pd.DataFrame(value)], ignore_index=False, axis=0)
                                                else:
                                                    dfsPSTH_per_sheet[key]=pd.DataFrame(value)


        ###########################################################################
                                ##### PREFERENCE #####
        ########################################################################### 
        Vig_cluster_df = Vig_cluster_dfO.copy()
        Vig_cluster_df = Vig_cluster_df[Vig_cluster_df['NeuronType'] == NrSubtype]
        Vig_cluster_df_Drug = Vig_cluster_df.copy()        
        AllUnits = Vig_cluster_df['Unit_ID'].unique()

        Vig_cluster_df_Drug = Vig_cluster_df_Drug[Vig_cluster_df_Drug['Drug'] == 'baseline']    
        AllBaselineUnits = Vig_cluster_df_Drug['Unit_ID'].unique()

        Cluster0units = Vig_cluster_df_Drug[Vig_cluster_df_Drug['ClusterHDBSCAN'] == 0]['Unit_ID'].unique()
        Cluster1units = Vig_cluster_df_Drug[Vig_cluster_df_Drug['ClusterHDBSCAN'] == 1]['Unit_ID'].unique()
        Cluster2units = Vig_cluster_df_Drug[Vig_cluster_df_Drug['ClusterHDBSCAN'] == 2]['Unit_ID'].unique()

        for drug in Drugs: 
            
            combined_df_Drug = combined_df.copy()
            try:
                combined_df_Drug = combined_df_Drug[combined_df_Drug['Drug'] == drug]
            except: 
                combined_df_Drug=combined_df_Drug
            
            folder_to_save2= f'{folder_to_save}/{drug}/'
            if NrSubtype=='L1NDNF_mice' and o==0 :
                os.makedirs(folder_to_save2)  

            listCluster=['AllUnits', 'AllBaselineUnits', 'Cluster0units', 'Cluster1units', 'Cluster2units'] if NrSubtype=='L1NDNF_mice' else ['AllUnits', 'AllBaselineUnits']

            # Print the names of the sheets and their corresponding DataFrames
            for List_name in listCluster:
                
                thelist=locals()[f'{List_name}']
                filtered_df = combined_df_Drug[combined_df_Drug['Unit_ID'].isin(thelist)]
                GroupList=['AllNr', 'PsNr', 'NoModNr', 'NgNr']

                if NrSubtype=='L1NDNF_mice' :#and o==0:
                    new_folder= f"{folder_to_save2}/{List_name}/{Osc}/"
                    os.makedirs(new_folder)

                ###########################################################################
                                        ##### AVERAGE PSTH #####
                ###########################################################################

                for coup in Coupling:
                    filenameOutAVG = f'{folder_to_save2}/{List_name}/{Osc}/{NrSubtype}_{coup}AVG&SEM.xlsx'
                    locals()[f'excel_writerAVG{coup}']= pd.ExcelWriter(filenameOutAVG)

                    filenameOutMean = f'{folder_to_save2}/{List_name}/{Osc}/{NrSubtype}_{coup}During.xlsx'
                    locals()[f'excel_writerMean{coup}']= pd.ExcelWriter(filenameOutMean)

                    filenameOutMean = f'{folder_to_save2}/{List_name}/{Osc}/{NrSubtype}_{coup}Baseline.xlsx'
                    locals()[f'excel_writerMeanBase{coup}']= pd.ExcelWriter(filenameOutMean)

                    filenameOutMean = f'{folder_to_save2}/{List_name}/{Osc}/{NrSubtype}_{coup}Before.xlsx'
                    locals()[f'excel_writerMeanBef{coup}']= pd.ExcelWriter(filenameOutMean)

                    filenameOutMean = f'{folder_to_save2}/{List_name}/{Osc}/{NrSubtype}_{coup}AllBefore.xlsx'
                    locals()[f'excel_writerMeanAllBef{coup}']= pd.ExcelWriter(filenameOutMean)

                    filenameOutMean = f'{folder_to_save2}/{List_name}/{Osc}/{NrSubtype}_{coup}PSTH.xlsx'
                    locals()[f'excel_writer{coup}']= pd.ExcelWriter(filenameOutMean)
                
                for data in Data:
                    
                    if data=='Ca':
                        filenameOutPN = f'{folder_to_save2}/{List_name}/{Osc}/{NrSubtype}_NrProportion.xlsx'
                        excel_writerPN= pd.ExcelWriter(filenameOutPN)
                    
                    NbNr=pd.DataFrame()

                    for coup in Coupling:  

                        BigArray=pd.DataFrame()
                        BigArrayBSL=pd.DataFrame()
                        AVGArray=pd.DataFrame()
                        AVGArrayBSL=pd.DataFrame()
                        AVGArraybefore=pd.DataFrame()
                        AVGArrayallbefore=pd.DataFrame()
                        AVGArrayBSLbefore=pd.DataFrame()
                        AVGArrayBSLallbefore=pd.DataFrame()
                        AVGArraybaseline=pd.DataFrame()
                        AVGArrayBSLbaseline=pd.DataFrame()
                        
                        for ctx in CTX:
                            
                            if f'{data}PSTH{Osc}_{coup}{ctx}{drug}' in locals(): #cause no ctx if uncoupled

                                df_per_sheet=locals()[f'{data}PSTH{Osc}_{coup}{ctx}{drug}']

                                #filenameOut = f'{folder_to_save2}/{List_name}/{Osc}/{NrSubtype}_{data}PSTH_{coup}{ctx}_{drug}.pkl' #keep each responses of each cells for all rec Osc
                                #with open(filenameOut, 'wb') as pickle_file:
                                #    pickle.dump(df_per_sheet, pickle_file)

                                if len(df_per_sheet.keys()): # not empty

                                    # if PSTH is full lenght
                                    arrayLen = 0
                                    for key, value in df_per_sheet.items():
                                        if value.size > 0:
                                            arrayLen = np.shape(value)[1]
                                            break  
                                    # if PSTH is 2 sec lenght    
                                    arrayLen=40
                                    
                                    for group in GroupList:
                                        locals()[f'{group}Array']=pd.DataFrame(columns=np.arange(0,arrayLen,1))
                                    
                                    for nr in df_per_sheet.keys():
                                    
                                        if df_per_sheet[nr].shape[0] >= 10 : # at least 10 oscillations recorded
                                            
                                            if arrayLen==40: # All Osc at 1second 
                                                if Osc== 'Spdl':
                                                    PSTH=df_per_sheet[nr].iloc[:, df_per_sheet[nr].shape[1] // 10 *4  : df_per_sheet[nr].shape[1] // 10 * 6] # -1 to 1 sec for Spdl (-5 to 5s)
                                                else:
                                                    PSTH=df_per_sheet[nr].iloc[:, df_per_sheet[nr].shape[1] // 6 *2  : df_per_sheet[nr].shape[1] // 6 * 4] # -1 to 1 sec for SWR & DS (-3 to 3s)
                                            else: # full lenght
                                                PSTH=df_per_sheet[nr]

                                            AVGtrace=np.nanmean(PSTH, axis=0)
                                            
                                            BaselineSTD=np.std(AVGtrace[:PSTH.shape[1]//4], axis=0)

                                            upperThrs=np.nanmean(AVGtrace[:PSTH.shape[1]//4], axis=0) + BaselineSTD*2
                                            lowerThrs=np.nanmean(AVGtrace[:PSTH.shape[1]//4], axis=0) - BaselineSTD*2

                                            AVGbefore=np.nanmean(AVGtrace[PSTH.shape[1]//4:PSTH.shape[1]//4*2], axis=0)
                                            AVGearly=np.nanmean(AVGtrace[PSTH.shape[1]//4*2:PSTH.shape[1]//4*3], axis=0)
                                            AVGlate=np.nanmean(AVGtrace[PSTH.shape[1]//4*3:PSTH.shape[1]//4*4], axis=0)
                                            
                                            if AVGearly > upperThrs:
                                                group='PsNr'
                                            elif AVGearly < lowerThrs:
                                                group='NgNr'
                                            else: 
                                                group='NoModNr'
                                            
                                            Array=locals()[f'{group}Array']
                                            Array.loc[nr]=AVGtrace
                                            Array=locals()[f'AllNrArray']
                                            Array.loc[nr]=AVGtrace

                                    for group in GroupList:

                                        ArrayO=locals()[f'{group}Array']

                                        if not ArrayO.isna().all().all():

                                            # Only keep the neurons belonging to the list (All units, NREM active, REM active, etc)
                                            present_indices = [idx for idx in thelist if idx in ArrayO.index]
                                            Array = ArrayO.loc[present_indices] 

                                            n=Array.shape[0]
                                            NbNr.loc[group,f'{ctx}{coup}']=n

                                            # Leave a blanck space for units not recorded in that condition
                                            missing_indexes = set(thelist) - set(Array.index)
                                            Array = Array.reindex(Array.index.union(missing_indexes))
                                            Array = Array.sort_index()

                                            ArrayO=Array.copy()
                                            mArray=Array.mean(axis=0)
                                            semArray = stats.sem(Array, axis=0, ddof=1, nan_policy='omit')
                                            icArray = norm.ppf((1 +  0.95) / 2) * (np.std(Array, axis=0) / np.sqrt(Array.shape[0]))
                                            #SmallArray=pd.DataFrame(np.transpose([mArray,semArray,icArray]), columns=[f'{ctx}{coup}{Osc} Mean', f'{ctx}{coup}{Osc} SEM', f'{ctx}{coup}{Osc} IC'])
                                            SmallArray=pd.DataFrame(np.transpose([mArray,semArray]), columns=[f'{group}{ctx}mean', f'{group}{ctx}sem'])
                                        
                                            BigArray=pd.concat([BigArray,SmallArray], axis=1)

                                            SecondHalf_columns = Array.iloc[:, Array.shape[1] //2:]
                                            mean_baseline = SecondHalf_columns.mean(axis=1)
                                            avgarray=pd.DataFrame(mean_baseline, columns=[f'{group}{ctx}'])
                                            AVGArray=pd.concat([AVGArray,avgarray], axis=1)

                                            FirstQuarter_columns = Array.iloc[:, :Array.shape[1] //4]
                                            mean_baseline = FirstQuarter_columns.mean(axis=1)
                                            avgarray=pd.DataFrame(mean_baseline, columns=[f'{group}{ctx}'])
                                            AVGArraybaseline=pd.concat([AVGArraybaseline,avgarray], axis=1)

                                            SecondQuarter_columns = Array.iloc[:, Array.shape[1]//4:Array.shape[1]//4*2]
                                            mean_baseline = SecondQuarter_columns.mean(axis=1)
                                            avgarray=pd.DataFrame(mean_baseline, columns=[f'{group}{ctx}'])
                                            AVGArraybefore=pd.concat([AVGArraybefore,avgarray], axis=1)

                                            FirstHalf_columns = Array.iloc[:, :Array.shape[1]//2]
                                            mean_baseline = FirstHalf_columns.mean(axis=1)
                                            avgarray=pd.DataFrame(mean_baseline, columns=[f'{group}{ctx}'])
                                            AVGArrayallbefore=pd.concat([AVGArrayallbefore,avgarray], axis=1)
                                            
                                            # Baseline signals
                                            baseline_columns = Array.iloc[:, :Array.shape[1] // 4]
                                            mean_baseline = baseline_columns.mean(axis=1)
                                            Array = Array.sub(mean_baseline, axis=0)
                                        
                                            excel_writer=locals()[f'excel_writer{coup}']
                                            Array.to_excel(excel_writer, sheet_name=f'{data}BSL_{group}{ctx}', index=True, header=False)
                                            ArrayO.to_excel(excel_writer, sheet_name=f'{data}{group}{ctx}', index=True, header=False)

                                            mArray=Array.mean(axis=0)
                                            semArray = stats.sem(Array, axis=0, ddof=1, nan_policy='omit')
                                            icArray = norm.ppf((1 +  0.95) / 2) * (np.std(Array, axis=0) / np.sqrt(Array.shape[0]))
                                            #SmallArray=pd.DataFrame(np.transpose([mArray,semArray,icArray]), columns=[f'{ctx}{coup}{Osc} Mean', f'{ctx}{coup}{Osc} SEM', f'{ctx}{coup}{Osc} IC'])
                                            SmallArray=pd.DataFrame(np.transpose([mArray,semArray]), columns=[f'{group}{ctx}mean', f'{group}{ctx}sem'])
                                            
                                            BigArrayBSL=pd.concat([BigArrayBSL,SmallArray], axis=1)

                                            SecondHalf_columns = Array.iloc[:, Array.shape[1]//2:]
                                            mean_baseline = SecondHalf_columns.mean(axis=1)
                                            avgarray=pd.DataFrame(mean_baseline, columns=[f'{group}{ctx}'])
                                            AVGArrayBSL=pd.concat([AVGArrayBSL,avgarray], axis=1)

                                            FirstQuarter_columns = Array.iloc[:, :Array.shape[1]//4]
                                            mean_baseline = FirstQuarter_columns.mean(axis=1)
                                            avgarray=pd.DataFrame(mean_baseline, columns=[f'{group}{ctx}'])
                                            AVGArrayBSLbaseline=pd.concat([AVGArrayBSLbaseline,avgarray], axis=1)

                                            SecondQuarter_columns = Array.iloc[:, Array.shape[1]//4:Array.shape[1]//4*2]
                                            mean_baseline = SecondQuarter_columns.mean(axis=1)
                                            avgarray=pd.DataFrame(mean_baseline, columns=[f'{group}{ctx}'])
                                            AVGArrayBSLbefore=pd.concat([AVGArrayBSLbefore,avgarray], axis=1)

                                            FirstHalf_columns = Array.iloc[:, :Array.shape[1]//2]
                                            mean_baseline = FirstHalf_columns.mean(axis=1)
                                            avgarray=pd.DataFrame(mean_baseline, columns=[f'{group}{ctx}'])
                                            AVGArrayBSLallbefore=pd.concat([AVGArrayBSLallbefore,avgarray], axis=1)
                                            
                        excel_writer=locals()[f'excel_writer{coup}']
                        excel_writer.close()
                        excel_writerAVGn=locals()[f'excel_writerAVG{coup}']
                        excel_writerMeann=locals()[f'excel_writerMean{coup}']
                        excel_writerMeannBase=locals()[f'excel_writerMeanBase{coup}']
                        excel_writerMeannBef=locals()[f'excel_writerMeanBef{coup}']
                        excel_writerMeannAllBef=locals()[f'excel_writerMeanAllBef{coup}']
                        
                        
                        BigArrayBSL = BigArrayBSL.sort_index(axis=1)
                        AVGArrayBSL = AVGArrayBSL.sort_index(axis=1)
                        BigArrayBSL.to_excel(excel_writerAVGn, sheet_name=f'BSL_{data}PSTH', index=True, header=True)
                        AVGArrayBSL.to_excel(excel_writerMeann, sheet_name=f'BSL_{data}PSTH', index=True, header=True)
                        AVGArrayBSLbaseline.to_excel(excel_writerMeannBase, sheet_name=f'BSL_{data}PSTH', index=True, header=True)
                        AVGArrayBSLbefore.to_excel(excel_writerMeannBef, sheet_name=f'BSL_{data}PSTH', index=True, header=True)
                        AVGArrayBSLallbefore.to_excel(excel_writerMeannAllBef, sheet_name=f'BSL_{data}PSTH', index=True, header=True)

                        BigArray = BigArray.sort_index(axis=1)
                        AVGArray = AVGArray.sort_index(axis=1)
                        BigArray.to_excel(excel_writerAVGn, sheet_name=f'{data}PSTH', index=True, header=True)
                        AVGArray.to_excel(excel_writerMeann, sheet_name=f'{data}PSTH', index=True, header=True)
                        AVGArraybaseline.to_excel(excel_writerMeannBase, sheet_name=f'{data}PSTH', index=True, header=True)
                        AVGArraybefore.to_excel(excel_writerMeannBef, sheet_name=f'{data}PSTH', index=True, header=True)
                        AVGArrayallbefore.to_excel(excel_writerMeannAllBef, sheet_name=f'{data}PSTH', index=True, header=True)

                    if data=='Ca':
                        try: 
                            NbNr.loc['PsNr %'] = round(NbNr.loc['PsNr'] / NbNr.loc['AllNr'] *100)
                            NbNr.loc['NoModNr %'] = round(NbNr.loc['NoModNr'] / NbNr.loc['AllNr'] *100)
                            NbNr.loc['NgNr %'] = round(NbNr.loc['NgNr'] / NbNr.loc['AllNr']*100)
                        except:                        
                            None
                        NbNr.to_excel(excel_writerPN, sheet_name=f'{data}', index=True, header=True)
                        excel_writerPN.close()

                for coup in Coupling:
                    excel_writerAVGn=locals()[f'excel_writerAVG{coup}']
                    excel_writerMeann=locals()[f'excel_writerMean{coup}']
                    excel_writerMeannBase=locals()[f'excel_writerMeanBase{coup}']
                    excel_writerMeannBef=locals()[f'excel_writerMeanBef{coup}']
                    excel_writerMeannAllBef=locals()[f'excel_writerMeanAllBef{coup}']
                    excel_writerAVGn.close()                   
                    excel_writerMeann.close() 
                    excel_writerMeannBase.close()     
                    excel_writerMeannBef.close()                   
                    excel_writerMeannAllBef.close()                   
            
    #######################
    # Propreties Osc
    #######################
    filenameOut = f'{folder_to_save}/{Osc}Propreties.xlsx'
    writer = pd.ExcelWriter(filenameOut)

    combined_dfOsc = combined_df.drop_duplicates(subset=f'{Osc}_ID', keep='first')

    if Osc== 'SWR':
        Count = pd.crosstab(index=combined_dfOsc['Mice'],columns=[combined_dfOsc['Drug'], combined_dfOsc[f'SWR_inside_Spdl']])
        AllOscSWRinside=pd.concat([AllOscSWRinside, Count], axis=0) 
        Count = pd.crosstab(index=combined_dfOsc['Mice'],columns=[combined_dfOsc['Drug'], combined_dfOsc[f'{Osc}Statut']])
        AllOscStatut=pd.concat([AllOscStatut, Count], axis=0)    
        OscDuration = combined_dfOsc.pivot_table(index='Mice', columns=[combined_dfOsc['Drug'],combined_dfOsc[f'{Osc}Statut']] , values=f'{Osc}Duration', aggfunc='mean')
        AllOscDuration=pd.concat([AllOscDuration, OscDuration], axis=0)       
    elif Osc== 'Spdl':
        Count = pd.crosstab(index=combined_dfOsc['Mice'],columns=[combined_dfOsc['Drug'], combined_dfOsc[f'SWR_inside_Spdl']])
        AllOscSWRinside=pd.concat([AllOscSWRinside, Count], axis=0) 
        GlobalSpindle = pd.crosstab(index=combined_dfOsc['Mice'],columns=[combined_dfOsc['Drug'], combined_dfOsc[f'GlobalSpindle'], combined_dfOsc[f'{Osc}StartLocation']])
        AllGlobalSpindle=pd.concat([AllGlobalSpindle, GlobalSpindle], axis=0)
        Count = pd.crosstab(index=combined_dfOsc['Mice'],columns=[combined_dfOsc['Drug'],combined_dfOsc['GlobalSpindle'], combined_dfOsc[f'{Osc}StartLocation']])
        AllOscStartLocation=pd.concat([AllOscStartLocation, Count], axis=0)
        OscDuration = combined_dfOsc.pivot_table(index='Mice', columns=[combined_dfOsc['Drug'],combined_dfOsc['GlobalSpindle'], combined_dfOsc[f'{Osc}StartLocation']] , values=f'{Osc}Duration', aggfunc='mean')
        AllOscDuration=pd.concat([AllOscDuration, OscDuration], axis=0)
        Count = pd.crosstab(index=combined_dfOsc['Mice'],columns=[combined_dfOsc['Drug'], combined_dfOsc[f'GlobalSpindle'],  combined_dfOsc[f'{Osc}StartLocation'], combined_dfOsc[f'{Osc}Statut']])
        AllOscCoupling=pd.concat([AllOscCoupling, Count], axis=0)
        Count = pd.crosstab(index=combined_dfOsc['Mice'],columns=[combined_dfOsc['Drug'], combined_dfOsc[f'GlobalSpindle'],  combined_dfOsc[f'{Osc}Statut']])
        AllOscStatut=pd.concat([AllOscStatut, Count], axis=0)
          
    AllOscDuration.to_excel(writer, sheet_name=f'MeanDuration')
    AllOscSWRinside.to_excel(writer, sheet_name=f'SWR_inside_Spdl')
    AllOscStatut.to_excel(writer, sheet_name=f'CouplingStatut')
    if Osc== 'Spdl':
        AllOscCoupling.to_excel(writer, sheet_name=f'CouplingPerSpdlLoc')    
        AllGlobalSpindle.to_excel(writer, sheet_name=f'GlobalSpindle')
        AllOscStartLocation.to_excel(writer, sheet_name=f'StartLocation')
       
    writer.close()

Spdl oscillations analysis...
1322 Total neurons in the cross-registration
4115 Spdl recorded in total
Spdl_CaPSTH_baseline
Spdl_CaPSTH_UnCoupledbaseline
Spdl_CaPSTH_Coupledbaseline
Spdl_CaPSTH_S1baseline
Spdl_CaPSTH_UnCoupledS1baseline
Spdl_CaPSTH_CoupledS1baseline
Spdl_CaPSTH_PFCbaseline
Spdl_CaPSTH_UnCoupledPFCbaseline
Spdl_CaPSTH_CoupledPFCbaseline
Spdl_CaPSTH_S1PFCbaseline
Spdl_CaPSTH_UnCoupledS1PFCbaseline
Spdl_CaPSTH_CoupledS1PFCbaseline
Spdl_CaPSTH_CGP
Spdl_CaPSTH_UnCoupledCGP
Spdl_CaPSTH_CoupledCGP
Spdl_CaPSTH_S1CGP
Spdl_CaPSTH_UnCoupledS1CGP
Spdl_CaPSTH_CoupledS1CGP
Spdl_CaPSTH_PFCCGP
Spdl_CaPSTH_UnCoupledPFCCGP
Spdl_CaPSTH_CoupledPFCCGP
Spdl_CaPSTH_S1PFCCGP
Spdl_CaPSTH_UnCoupledS1PFCCGP
Spdl_CaPSTH_CoupledS1PFCCGP
Spdl_SpPSTH_baseline
Spdl_SpPSTH_UnCoupledbaseline
Spdl_SpPSTH_Coupledbaseline
Spdl_SpPSTH_S1baseline
Spdl_SpPSTH_UnCoupledS1baseline
Spdl_SpPSTH_CoupledS1baseline
Spdl_SpPSTH_PFCbaseline
Spdl_SpPSTH_UnCoupledPFCbaseline
Spdl_SpPSTH_CoupledPFCbaseline
Spdl_SpPSTH_S